In [8]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

In [9]:
patents = pd.read_csv("google-patent-data.csv")
patents.head(2)

,id,title,grant date,result link,representative figure link
0,US-PP36186-P2,Interspecific prunus tree named ‘Sweet Pixie 6’,2024-10-08,https://patents.google.com/patent/USPP36186P2/en,https://patentimages.storage.googleapis.com/45...
1,US-PP35196-P2,Interspecific tree named ‘Lavish Lacey’,2023-05-30,https://patents.google.com/patent/USPP35196P2/en,https://patentimages.storage.googleapis.com/34...


In [10]:
urls = patents['result link'].tolist()
urls

['https://patents.google.com/patent/USPP36186P2/en',
 'https://patents.google.com/patent/USPP35196P2/en',
 'https://patents.google.com/patent/USPP35178P2/en',
 'https://patents.google.com/patent/USPP34290P2/en',
 'https://patents.google.com/patent/USPP34426P2/en',
 'https://patents.google.com/patent/USPP34087P2/en',
 'https://patents.google.com/patent/USPP35352P2/en',
 'https://patents.google.com/patent/USPP33622P2/en',
 'https://patents.google.com/patent/USPP33444P2/en',
 'https://patents.google.com/patent/USPP32878P2/en',
 'https://patents.google.com/patent/USPP32851P2/en',
 'https://patents.google.com/patent/USPP32703P2/en',
 'https://patents.google.com/patent/USPP31823P2/en',
 'https://patents.google.com/patent/USPP29881P2/en',
 'https://patents.google.com/patent/USPP29882P2/en',
 'https://patents.google.com/patent/USPP30010P2/en',
 'https://patents.google.com/patent/USPP29366P2/en',
 'https://patents.google.com/patent/USPP28137P2/en',
 'https://patents.google.com/patent/USPP28151P

In [11]:
munsell_to_hex = {
    '10R 2/4': '#4e2521',
    '10R 2/6': '#581e1a',
    '10R 3/10': '#892210',
    '10R 3/6': '#743529',
    '10R 3/8': '#7f2d1d',
    '10R 4/8': '#9b4832',
    '10R 5/12': '#ce5427',
    '10R 5/14': '#d74c12',
    '10R 6/12': '#ec7042',
    '10R 6/8': '#d37f62',
    '10Y 5/6': '#817e2a',
    '10Y 6/4': '#9a9760',
    '10Y 6/6': '#9c9843',
    '10Y 7/4': '#b5b278',
    '10Y 7/6': '#b7b35d',
    '10Y 8/4': '#d0cd91',
    '10Y 8/6': '#d2ce74',
    '10Y 8/8': '#d4cf56',
    '10Y 9/2': '#e9e7c4',
    '10YR 4/6': '#805920',
    '10YR 5/6': '#805920',
    '10YR 6/10': '#c58910',
    '10YR 6/8': '#c08b36',
    '10YR 7/10': '#e2a433',
    '10YR 7/12': '#E6A000',
    '10YR 7/6': '#d3a868',
    '10YR 7/8': '#dba650',
    '10YR 8/10': '#FDBD4E',
    '10YR 8/12': None,
    '10YR 8/2': '#d8c8b1',
    '10YR 8/4': '#e4c599',
    '10YR 8/6': '#eec382',
    '10YR 8/8': '#f8c168',
    '10YR 9/1': '#eae2d5',
    '10YR 9/2': '#f4e3ca',
    '2.5GY 6/4': '#939963',
    '2.5GY 7/6': '#adb561',
    '2.5GY 8/6': '#c9d078',
    '2.5R 2/2': '#412b2f',
    '2.5R 2/4': '#4d252e',
    '2.5R 2/6': '#571d2e',
    '2.5R 2/8': '#610e2e',
    '2.5R 3/10': '#8c1637',
    '2.5R 3/6': '#75333c',
    '2.5R 3/8': '#812739',
    '2.5R 4/10': '#a73b4d',
    '2.5R 4/8': '#9c4550',
    '2.5R 5/8': '#b86068',
    '2.5Y 5/10': None,
    '2.5Y 6/10': None,
    '2.5Y 6/4': '#a99264',
    '2.5Y 6/6': '#b19049',
    '2.5Y 7/10': '#d9a822',
    '2.5Y 7/12': None,
    '2.5Y 7/4': '#c4ac7c',
    '2.5Y 7/6': '#cdab63',
    '2.5Y 7/8': '#d3aa48',
    '2.5Y 8/10': '#f5c342',
    '2.5Y 8/4': '#dfc895',
    '2.5Y 8/6': '#e7c67b',
    '2.5Y 8/8': '#efc561',
    '2.5Y 9/2': '#f1e4c7',
    '2.5Y 9/4': '#fbe3ac',
    '2.5YR 4/6': '#8d5136',
    '2.5YR 5/10': '#bf6029',
    '2.5YR 5/14': None,
    '2.5YR 5/8': '#b5663d',
    '2.5YR 6/10': '#dc7b45',
    '2.5YR 6/12': '#e6762e',
    '2.5YR 6/6': '#c58667',
    '2.5YR 6/8': '#d18157',
    '5GY 9/4': '#dfecab',
    '5R 2/2': '#422b2d',
    '5R 2/4': '#4d252a',
    '5R 2/6': '#581d28',
    '5R 2/8': '#630d25',
    '5R 3/10': '#8d182b',
    '5R 3/4': '#6a3b3b',
    '5R 3/6': '#753336',
    '5R 3/8': '#812930',
    '5R 4/10': '#a83c40',
    '5R 4/12': '#b22f3a',
    '5R 4/2': '#735b5a',
    '5R 4/6': '#904e4c',
    '5R 4/8': '#9d4546',
    '5R 6/10': '#e27471',
    '5R 7/6': '#e19e99',
    '5R 9/2': '#f3e1df',
    '5Y 6/6': '#aa9345',
    '5Y 6/8': '#ae9322',
    '5Y 7/10': '#cdad0e',
    '5Y 7/6': '#c4ae5e',
    '5Y 7/8': '#c9ae40',
    '5Y 8/10': '#e8c935',
    '5Y 8/2': '#d2caac',
    '5Y 8/4': '#d8ca92',
    '5Y 8/6': '#dfca76',
    '5Y 8/8': '#e4c959',
    '5Y 9/2': '#eee5c5',
    '5Y 9/4': '#f4e6a9',
    '5Y 9/6': '#fae58d',
    '5YR 5/10': '#b86619',
    '5YR 5/12': None,
    '5YR 5/8': '#b06931',
    '5YR 6/10': '#d68034',
    '5YR 6/12': '#dd7c14',
    '5YR 6/14': None,
    '5YR 6/8': '#cd844b',
    '5YR 7/10': '#f39b4f',
    '5YR 7/12': '#fc9633',
    '5YR 7/8': '#e99f65',
    '6.25R 4/12': None,
    '6.25YR 6/12': None,
    '6.25YR 7/12': None,
    '6.25YR 7/14': None,
    '7.5R 2/2': '#422b2b',
    '7.5R 2/4': '#4e2526',
    '7.5R 2/6': '#581d21',
    '7.5R 2/8': '#62101c',
    '7.5R 3/10': '#8b1d1e',
    '7.5R 3/12': None,
    '7.5R 3/4': '#6a3b37',
    '7.5R 3/6': '#75342f',
    '7.5R 3/8': '#812a27',
    '7.5R 4/10': '#a83d34',
    '7.5R 4/12': '#b2322b',
    '7.5R 4/14': '#bc2120',
    '7.5R 4/16': None,
    '7.5R 5/10': '#c6594a',
    '7.5R 5/16': '#e6352a',
    '7.5R 5/8': '#b96154',
    '7.5R 6/8': '#d47d6e',
    '7.5RP 2/6': '#531e38',
    '7.5RP 2/8': '#5d113b',
    '7.5Y 5/8': None,
    '7.5Y 6/4': '#9f9660',
    '7.5Y 6/6': '#a39642',
    '7.5Y 7/4': '#b9b078',
    '7.5Y 7/8': '#c0b13d',
    '7.5Y 8/2': '#d0cbab',
    '7.5Y 8/4': '#d4cb91',
    '7.5Y 8/6': '#d8cc75',
    '7.5Y 8/8': '#dccc56',
    '7.5Y 9/4': '#f0e7a8',
    '7.5Y 9/6': '#f4e88b',
    '7.5YR 5/10': None,
    '7.5YR 5/8': '#aa6d26',
    '7.5YR 6/10': '#ce8524',
    '7.5YR 6/12': None,
    '7.5YR 6/14': None,
    '7.5YR 6/8': '#c78741',
    '7.5YR 7/10': '#eb9f41',
    '7.5YR 7/12': '#f39c1c',
    '7.5YR 7/14': None,
    '7.5YR 7/6': '#d9a570',
    '8.75YR 7/12': None
}

In [12]:
# test code

all_patents = []

munsell_pattern = re.compile(r'\d+(\.\d+)?[A-Z]{1,2}\s\d+\/\d+')
rhs_pattern = re.compile(r'\d{1,2}-[A-Z]-\d')

for url in urls [:2]:
    response = requests.get(url)
    response.encoding = 'utf-8'
    doc = BeautifulSoup(response.text, 'html.parser')

    description = doc.find('section', itemprop='description')

    text = description.get_text(separator='\n')
    lines = [line.strip() for line in text.splitlines() if line.strip()]

    current_section = None
    patent_data = {
        'url': url,
        'flesh': {},
        'skin': {}
    }

    for i, line in enumerate(lines):
        if line.startswith("Flesh:"):
            current_section = 'flesh'
        elif line.startswith("Skin:"):
            current_section = 'skin'
        elif line.endswith(":"):
            current_section = None
        
        if current_section is None:
            continue
        
        if re.match(r'^Color[\s\.\—\-–]*$', line) and i + 1 < len(lines):
            full_color = line + " " + lines[i + 1]
        elif line.startswith("Color"):
            full_color = line
        else:
            full_color = None

        if full_color:
            cleaned_color = re.sub(r'[()—–]', '', full_color)
            munsell_codes = [m.group(0) for m in munsell_pattern.finditer(cleaned_color)]
            rhs_codes = [m.group(0) for m in rhs_pattern.finditer(cleaned_color)]

            hex_codes = []
            for code in munsell_codes:
                if code in munsell_to_hex:
                    hex_codes.append(munsell_to_hex.get(code))

            patent_data[current_section]['color'] = full_color.strip()
            patent_data[current_section]['munsell'] = munsell_codes
            patent_data[current_section]['hex'] = hex_codes
            patent_data[current_section]['rhs'] = rhs_codes
        
        if re.match(r'^Brix[\s\.\—\-–]*$', line) and i + 1 < len(lines):
            full_brix = line + " " + lines[i + 1]
        elif line.startswith("Brix"):
            full_brix = line
        else:
            full_brix = None

        if full_brix:
            brix_match = re.search(r'(\d+(\.\d+)?)', full_brix)
            brix_value = brix_match.group(1) if brix_match else None

            patent_data[current_section] = {
                'brix_text': full_brix.strip(),
                'brix_value': brix_value
            }

            patent_data[current_section]['brix_text'] = full_brix.strip()
            patent_data[current_section]['brix_value'] = brix_value
    
    all_patents.append(patent_data)

    print('------')
    print(url)
    for section in ['flesh', 'skin']:
        if section in patent_data and patent_data[section]:
            print(f"{section}: {patent_data[section]['color']}")
            print(f"Munsell: {patent_data[section]['munsell']}")
            print(f"Hex: {patent_data[section]['hex']}")
            print(f"RHS: {patent_data[section]['rhs']}")
            if section == 'flesh' and 'brix_text' in patent_data['flesh']:
                print(f"Brix: {patent_data[section]['brix_text']}")
                print(f"Brix Value: {patent_data[section]['brix_value']}")

------
https://patents.google.com/patent/USPP36186P2/en
flesh: Color .—Varies from 2.5Y 8/10 to 5Y 8/8.
Munsell: ['2.5Y 8/10', '5Y 8/8']
Hex: ['#f5c342', '#e4c959']
RHS: []
Brix: Brix .—Average Brix 26.9°, varies slightly with amount of fruit per tree and climatic conditions.
Brix Value: 26.9
skin: Color .—Ground color varies from 7.5Y 8/6 to 10Y 8/4.-Overspread with 5R 2/4 to 7.5R 3/8. Very small, randomly spaced areas of ground color exposed to give a speckling pattern to surface areas.
Munsell: ['7.5Y 8/6', '10Y 8/4', '5R 2/4', '7.5R 3/8']
Hex: ['#d8cc75', '#d0cd91', '#4d252a', '#812a27']
RHS: []
------
https://patents.google.com/patent/USPP35196P2/en
flesh: Color .—Varies from 5R 3/8 to 5R 3/6.
Munsell: ['5R 3/8', '5R 3/6']
Hex: ['#812930', '#753336']
RHS: []
Brix: Brix .—Average Brix 16.5°, varies slightly with amount of fruit per tree and climatic conditions.
Brix Value: 16.5
skin: Color .—Ground color varies from 2.5Y 8.5/4 to 5Y 8/4. Overspread with 5R 3/6 to 7.5R 3/6. Very sma

In [13]:
all_patents = []

munsell_pattern = re.compile(r'\d+(\.\d+)?[A-Z]{1,2}\s\d+\/\d+')
rhs_pattern = re.compile(r'\d{1,2}-[A-Z]-\d')

for url in urls:
    response = requests.get(url)
    response.encoding = 'utf-8'
    doc = BeautifulSoup(response.text, 'html.parser')

    description = doc.find('section', itemprop='description')

    text = description.get_text(separator='\n')
    lines = [line.strip() for line in text.splitlines() if line.strip()]

    current_section = None
    patent_data = {
        'url': url,
        'flesh': {},
        'skin': {}
    }

    for i, line in enumerate(lines):
        if line.startswith("Flesh:"):
            current_section = 'flesh'
        elif line.startswith("Skin:"):
            current_section = 'skin'
        elif line.endswith(":"):
            current_section = None
        
        if current_section is None:
            continue
        
        if re.match(r'^Color[\s\.\—\-–]*$', line) and i + 1 < len(lines):
            full_color = line + " " + lines[i + 1]
        elif line.startswith("Color"):
            full_color = line
        else:
            full_color = None

        if full_color:
            cleaned_color = re.sub(r'[()—–]', '', full_color)
            munsell_codes = [m.group(0) for m in munsell_pattern.finditer(cleaned_color)]
            rhs_codes = [m.group(0) for m in rhs_pattern.finditer(cleaned_color)]

            hex_codes = []
            for code in munsell_codes:
                if code in munsell_to_hex:
                    hex_codes.append(munsell_to_hex.get(code))

            patent_data[current_section]['color'] = full_color.strip()
            patent_data[current_section]['munsell'] = munsell_codes
            patent_data[current_section]['hex'] = hex_codes
            patent_data[current_section]['rhs'] = rhs_codes
        
        if re.match(r'^Brix[\s\.\—\-–]*$', line) and i + 1 < len(lines):
            full_brix = line + " " + lines[i + 1]
        elif line.startswith("Brix"):
            full_brix = line
        else:
            full_brix = None

        if full_brix:
            brix_match = re.search(r'(\d+(\.\d+)?)', full_brix)
            brix_value = brix_match.group(1) if brix_match else None

            patent_data[current_section] = {
                'brix_text': full_brix.strip(),
                'brix_value': brix_value
            }

            patent_data[current_section]['brix_text'] = full_brix.strip()
            patent_data[current_section]['brix_value'] = brix_value
    
    all_patents.append(patent_data)

    print('------')
    print(url)
    for section in ['flesh', 'skin']:
        if section in patent_data and patent_data[section]:
            print(f"{section}: {patent_data[section]['color']}")
            print(f"Munsell: {patent_data[section]['munsell']}")
            print(f"Hex: {patent_data[section]['hex']}")
            print(f"RHS: {patent_data[section]['rhs']}")
            if section == 'flesh' and 'brix_text' in patent_data['flesh']:
                print(f"Brix: {patent_data[section]['brix_text']}")
                print(f"Brix Value: {patent_data[section]['brix_value']}")

------
https://patents.google.com/patent/USPP36186P2/en
flesh: Color .—Varies from 2.5Y 8/10 to 5Y 8/8.
Munsell: ['2.5Y 8/10', '5Y 8/8']
Hex: ['#f5c342', '#e4c959']
RHS: []
Brix: Brix .—Average Brix 26.9°, varies slightly with amount of fruit per tree and climatic conditions.
Brix Value: 26.9
skin: Color .—Ground color varies from 7.5Y 8/6 to 10Y 8/4.-Overspread with 5R 2/4 to 7.5R 3/8. Very small, randomly spaced areas of ground color exposed to give a speckling pattern to surface areas.
Munsell: ['7.5Y 8/6', '10Y 8/4', '5R 2/4', '7.5R 3/8']
Hex: ['#d8cc75', '#d0cd91', '#4d252a', '#812a27']
RHS: []
------
https://patents.google.com/patent/USPP35196P2/en
flesh: Color .—Varies from 5R 3/8 to 5R 3/6.
Munsell: ['5R 3/8', '5R 3/6']
Hex: ['#812930', '#753336']
RHS: []
Brix: Brix .—Average Brix 16.5°, varies slightly with amount of fruit per tree and climatic conditions.
Brix Value: 16.5
skin: Color .—Ground color varies from 2.5Y 8.5/4 to 5Y 8/4. Overspread with 5R 3/6 to 7.5R 3/6. Very sma

In [ ]:
# refactored version with functions

munsell_pattern = re.compile(r'\d+(\.\d+)?[A-Z]{1,2}\s\d+\/\d+')
rhs_pattern = re.compile(r'\d{1,2}-[A-Z]-\d')

def get_patent_text(url):
    response = requests.get(url)
    response.encoding = 'utf-8'
    doc = BeautifulSoup(response.text, 'html.parser')
    description = doc.find('section', itemprop='description')
    text = description.get_text(separator='\n')
    return [line.strip() for line in text.splitlines() if line.strip()]

def extract_color_info(lines, i):
    line = lines[i]
    if re.match(r'^Color[\s\.\—\-–]*$', line) and i + 1 < len(lines):
        full_color = line + " " + lines[i + 1]
    elif line.startswith("Color"):
        full_color = line
    else:
        return None

    cleaned_color = re.sub(r'[()—–]', '', full_color)
    munsell_codes = [m.group(0) for m in munsell_pattern.finditer(cleaned_color)]
    rhs_codes = [m.group(0) for m in rhs_pattern.finditer(cleaned_color)]

    hex_codes = []
    for code in munsell_codes:
        if code in munsell_to_hex:
            hex_codes.append(munsell_to_hex[code])

    return {
        'color': full_color.strip(),
        'munsell': munsell_codes,
        'rhs': rhs_codes,
        'hex': hex_codes
    }

def extract_brix_info(lines, i):
    line = lines[i]
    if re.match(r'^Brix[\s\.\—\-–]*$', line) and i + 1 < len(lines):
        full_brix = line + " " + lines[i + 1]
    elif line.startswith("Brix"):
        full_brix = line
    else:
        return None

    brix_match = re.search(r'(\d+(\.\d+)?)', full_brix)
    brix_value = brix_match.group(1) if brix_match else None

    return {
        'brix_text': full_brix.strip(),
        'brix_value': brix_value
    }

def parse_patent(url):
    lines = get_patent_text(url)
    current_section = None
    patent_data = {
        'url': url,
        'flesh': {},
        'skin': {}
    }

    for i, line in enumerate(lines):
        if line.startswith("Flesh:"):
            current_section = 'flesh'
        elif line.startswith("Skin:"):
            current_section = 'skin'
        elif line.endswith(":"):
            current_section = None
        
        if current_section is None:
            continue

        color_info = extract_color_info(lines, i)
        if color_info:
            patent_data[current_section].update(color_info)

        brix_info = extract_brix_info(lines, i)
        if brix_info:
            patent_data[current_section].update(brix_info)

    return patent_data


all_patents = []

for url in urls:
    patent = parse_patent(url)
    all_patents.append(patent)

    print('------')
    print(url)
    for section in ['flesh', 'skin']:
        if patent[section]:
            print(f"{section}: {patent[section].get('color')}")
            print(f"Munsell: {patent[section].get('munsell')}")
            print(f"Hex: {patent[section].get('hex')}")
            print(f"RHS: {patent[section].get('rhs')}")
            if 'brix_text' in patent[section]:
                print(f"Brix: {patent[section]['brix_text']}")
                print(f"Brix Value: {patent[section]['brix_value']}")

In [14]:
len(all_patents)

100

In [15]:
all_patents[:3]

[{'url': 'https://patents.google.com/patent/USPP36186P2/en',
  'flesh': {'brix_text': 'Brix .—Average Brix 26.9°, varies slightly with amount of fruit per tree and climatic conditions.',
   'brix_value': '26.9',
   'color': 'Color .—Varies from 2.5Y 8/10 to 5Y 8/8.',
   'munsell': ['2.5Y 8/10', '5Y 8/8'],
   'hex': ['#f5c342', '#e4c959'],
   'rhs': []},
  'skin': {'color': 'Color .—Ground color varies from 7.5Y 8/6 to 10Y 8/4.-Overspread with 5R 2/4 to 7.5R 3/8. Very small, randomly spaced areas of ground color exposed to give a speckling pattern to surface areas.',
   'munsell': ['7.5Y 8/6', '10Y 8/4', '5R 2/4', '7.5R 3/8'],
   'hex': ['#d8cc75', '#d0cd91', '#4d252a', '#812a27'],
   'rhs': []}},
 {'url': 'https://patents.google.com/patent/USPP35196P2/en',
  'flesh': {'brix_text': 'Brix .—Average Brix 16.5°, varies slightly with amount of fruit per tree and climatic conditions.',
   'brix_value': '16.5',
   'color': 'Color .—Varies from 5R 3/8 to 5R 3/6.',
   'munsell': ['5R 3/8', '5R 

In [16]:
unique_munsell = set()
unique_rhs = set()

for patent in all_patents:
    for section in ['flesh', 'skin']:
        section_data = patent.get(section, {})
        unique_munsell.update(section_data.get('munsell', []))
        unique_rhs.update(section_data.get('rhs', []))

print("Unique Munsell codes:")
for code in sorted(unique_munsell):
    print(code)

print("------")

print("Unique RHS codes:")
for code in sorted(unique_rhs):
    print(code)

Unique Munsell codes:
10R 2/4
10R 2/6
10R 3/10
10R 3/6
10R 3/8
10R 4/8
10R 5/12
10R 5/14
10R 6/12
10R 6/8
10Y 5/6
10Y 6/4
10Y 6/6
10Y 7/4
10Y 7/6
10Y 8/4
10Y 8/6
10Y 8/8
10Y 9/2
10YR 4/6
10YR 5/6
10YR 6/10
10YR 6/8
10YR 7/10
10YR 7/12
10YR 7/6
10YR 7/8
10YR 8/10
10YR 8/12
10YR 8/2
10YR 8/4
10YR 8/6
10YR 8/8
10YR 9/1
10YR 9/2
2.5GY 6/4
2.5GY 7/6
2.5GY 8/6
2.5R 2/2
2.5R 2/4
2.5R 2/6
2.5R 2/8
2.5R 3/10
2.5R 3/6
2.5R 3/8
2.5R 4/10
2.5R 4/8
2.5R 5/8
2.5Y 5/10
2.5Y 6/10
2.5Y 6/4
2.5Y 6/6
2.5Y 7/10
2.5Y 7/12
2.5Y 7/4
2.5Y 7/6
2.5Y 7/8
2.5Y 8/10
2.5Y 8/4
2.5Y 8/6
2.5Y 8/8
2.5Y 9/2
2.5Y 9/4
2.5YR 4/6
2.5YR 5/10
2.5YR 5/14
2.5YR 5/8
2.5YR 6/10
2.5YR 6/12
2.5YR 6/6
2.5YR 6/8
5GY 9/4
5R 2/2
5R 2/4
5R 2/6
5R 2/8
5R 3/10
5R 3/4
5R 3/6
5R 3/8
5R 4/10
5R 4/12
5R 4/2
5R 4/6
5R 4/8
5R 6/10
5R 7/6
5R 9/2
5Y 6/6
5Y 6/8
5Y 7/10
5Y 7/6
5Y 7/8
5Y 8/10
5Y 8/2
5Y 8/4
5Y 8/6
5Y 8/8
5Y 9/2
5Y 9/4
5Y 9/6
5YR 5/10
5YR 5/12
5YR 5/8
5YR 6/10
5YR 6/12
5YR 6/14
5YR 6/8
5YR 7/10
5YR 7/12
5YR 7/8
6.25R 4/12
6.25YR 6/12


In [17]:
raw_munsell_text = """
10R 2/4
10R 2/6
10R 3/10
10R 3/6
10R 3/8
10R 4/8
10R 5/12
10R 5/14
10R 6/12
10R 6/8
10Y 5/6
10Y 6/4
10Y 6/6
10Y 7/4
10Y 7/6
10Y 8/4
10Y 8/6
10Y 8/8
10Y 9/2
10YR 4/6
10YR 5/6
10YR 6/10
10YR 6/8
10YR 7/10
10YR 7/12
10YR 7/6
10YR 7/8
10YR 8/10
10YR 8/12
10YR 8/2
10YR 8/4
10YR 8/6
10YR 8/8
10YR 9/1
10YR 9/2
2.5GY 6/4
2.5GY 7/6
2.5GY 8/6
2.5R 2/2
2.5R 2/4
2.5R 2/6
2.5R 2/8
2.5R 3/10
2.5R 3/6
2.5R 3/8
2.5R 4/10
2.5R 4/8
2.5R 5/8
2.5Y 5/10
2.5Y 6/10
2.5Y 6/4
2.5Y 6/6
2.5Y 7/10
2.5Y 7/12
2.5Y 7/4
2.5Y 7/6
2.5Y 7/8
2.5Y 8/10
2.5Y 8/4
2.5Y 8/6
2.5Y 8/8
2.5Y 9/2
2.5Y 9/4
2.5YR 4/6
2.5YR 5/10
2.5YR 5/14
2.5YR 5/8
2.5YR 6/10
2.5YR 6/12
2.5YR 6/6
2.5YR 6/8
5GY 9/4
5R 2/2
5R 2/4
5R 2/6
5R 2/8
5R 3/10
5R 3/4
5R 3/6
5R 3/8
5R 4/10
5R 4/12
5R 4/2
5R 4/6
5R 4/8
5R 6/10
5R 7/6
5R 9/2
5Y 6/6
5Y 6/8
5Y 7/10
5Y 7/6
5Y 7/8
5Y 8/10
5Y 8/2
5Y 8/4
5Y 8/6
5Y 8/8
5Y 9/2
5Y 9/4
5Y 9/6
5YR 5/10
5YR 5/12
5YR 5/8
5YR 6/10
5YR 6/12
5YR 6/14
5YR 6/8
5YR 7/10
5YR 7/12
5YR 7/8
6.25R 4/12
6.25YR 6/12
6.25YR 7/12
6.25YR 7/14
7.5R 2/2
7.5R 2/4
7.5R 2/6
7.5R 2/8
7.5R 3/10
7.5R 3/12
7.5R 3/4
7.5R 3/6
7.5R 3/8
7.5R 4/10
7.5R 4/12
7.5R 4/14
7.5R 4/16
7.5R 5/10
7.5R 5/16
7.5R 5/8
7.5R 6/8
7.5RP 2/6
7.5RP 2/8
7.5Y 5/8
7.5Y 6/4
7.5Y 6/6
7.5Y 7/4
7.5Y 7/8
7.5Y 8/2
7.5Y 8/4
7.5Y 8/6
7.5Y 8/8
7.5Y 9/4
7.5Y 9/6
7.5YR 5/10
7.5YR 5/8
7.5YR 6/10
7.5YR 6/12
7.5YR 6/14
7.5YR 6/8
7.5YR 7/10
7.5YR 7/12
7.5YR 7/14
7.5YR 7/6
8.75YR 7/12
"""

munsell_codes = [line.strip() for line in raw_munsell_text.strip().splitlines()]

munsell_to_hex = dict.fromkeys(munsell_codes, None)

list(munsell_to_hex.keys())[:5]

['10R 2/4', '10R 2/6', '10R 3/10', '10R 3/6', '10R 3/8']

In [18]:
from prettyprinter import pprint

pprint(munsell_to_hex)

{
    '10R 2/4': None,
    '10R 2/6': None,
    '10R 3/10': None,
    '10R 3/6': None,
    '10R 3/8': None,
    '10R 4/8': None,
    '10R 5/12': None,
    '10R 5/14': None,
    '10R 6/12': None,
    '10R 6/8': None,
    '10Y 5/6': None,
    '10Y 6/4': None,
    '10Y 6/6': None,
    '10Y 7/4': None,
    '10Y 7/6': None,
    '10Y 8/4': None,
    '10Y 8/6': None,
    '10Y 8/8': None,
    '10Y 9/2': None,
    '10YR 4/6': None,
    '10YR 5/6': None,
    '10YR 6/10': None,
    '10YR 6/8': None,
    '10YR 7/10': None,
    '10YR 7/12': None,
    '10YR 7/6': None,
    '10YR 7/8': None,
    '10YR 8/10': None,
    '10YR 8/12': None,
    '10YR 8/2': None,
    '10YR 8/4': None,
    '10YR 8/6': None,
    '10YR 8/8': None,
    '10YR 9/1': None,
    '10YR 9/2': None,
    '2.5GY 6/4': None,
    '2.5GY 7/6': None,
    '2.5GY 8/6': None,
    '2.5R 2/2': None,
    '2.5R 2/4': None,
    '2.5R 2/6': None,
    '2.5R 2/8': None,
    '2.5R 3/10': None,
    '2.5R 3/6': None,
    '2.5R 3/8': None,
    '2.5R 4/10

using [this color conversion tool](https://www.andrewwerth.com/color/) and [this one](https://pteromys.melonisland.net/munsell/)

In [19]:
raw_rhs_text = """
1-A-2
10-A-4
10-B-8
10-C-8
10-D-8
10-E-8
10-F-8
11-A-4
11-B-8
11-C-8
11-D-8
11-E-8
11-F-8
12-C-8
12-D-8
12-E-8
12-F-7
12-F-8
13-F-3
14-E-5
14-F-3
15-F-4
2-A-1
2-A-2
2-A-3
2-B-2
3-A-1
3-A-2
3-A-3
3-B-8
3-C-8
4-A-2
4-A-3
4-A-4
4-A-5
4-A-6
4-A-7
4-B-4
4-B-5
4-B-7
4-B-8
4-C-7
5-A-3
5-A-4
5-A-5
5-A-6
5-A-7
5-A-8
5-B-4
5-B-5
5-B-6
5-B-8
5-C-6
5-D-6
6-A-3
6-A-6
6-A-7
6-B-8
6-C-6
6-C-8
6-D-6
6-D-8
7-A-4
7-B-7
7-C-8
8-C-8
8-D-8
9-A-8
9-D-8
9-E-7
"""

rhs_codes = [line.strip() for line in raw_rhs_text.strip().splitlines()]

rhs_dict = dict.fromkeys(rhs_codes, None)

list(rhs_dict.keys())[:5]

['1-A-2', '10-A-4', '10-B-8', '10-C-8', '10-D-8']

In [20]:
pprint(rhs_dict)

{
    '1-A-2': None,
    '10-A-4': None,
    '10-B-8': None,
    '10-C-8': None,
    '10-D-8': None,
    '10-E-8': None,
    '10-F-8': None,
    '11-A-4': None,
    '11-B-8': None,
    '11-C-8': None,
    '11-D-8': None,
    '11-E-8': None,
    '11-F-8': None,
    '12-C-8': None,
    '12-D-8': None,
    '12-E-8': None,
    '12-F-7': None,
    '12-F-8': None,
    '13-F-3': None,
    '14-E-5': None,
    '14-F-3': None,
    '15-F-4': None,
    '2-A-1': None,
    '2-A-2': None,
    '2-A-3': None,
    '2-B-2': None,
    '3-A-1': None,
    '3-A-2': None,
    '3-A-3': None,
    '3-B-8': None,
    '3-C-8': None,
    '4-A-2': None,
    '4-A-3': None,
    '4-A-4': None,
    '4-A-5': None,
    '4-A-6': None,
    '4-A-7': None,
    '4-B-4': None,
    '4-B-5': None,
    '4-B-7': None,
    '4-B-8': None,
    '4-C-7': None,
    '5-A-3': None,
    '5-A-4': None,
    '5-A-5': None,
    '5-A-6': None,
    '5-A-7': None,
    '5-A-8': None,
    '5-B-4': None,
    '5-B-5': None,
    '5-B-6': None,
    '5-B

In [21]:
rhs_dict = {
    '1-A-2': None,
    '10-A-4': None,
    '10-B-8': None,
    '10-C-8': None,
    '10-D-8': None,
    '10-E-8': None,
    '10-F-8': None,
    '11-A-4': None,
    '11-B-8': None,
    '11-C-8': None,
    '11-D-8': None,
    '11-E-8': None,
    '11-F-8': None,
    '12-C-8': None,
    '12-D-8': None,
    '12-E-8': None,
    '12-F-7': None,
    '12-F-8': None,
    '13-F-3': None,
    '14-E-5': None,
    '14-F-3': None,
    '15-F-4': None,
    '2-A-1': None,
    '2-A-2': None,
    '2-A-3': None,
    '2-B-2': None,
    '3-A-1': None,
    '3-A-2': None,
    '3-A-3': None,
    '3-B-8': None,
    '3-C-8': None,
    '4-A-2': None,
    '4-A-3': None,
    '4-A-4': None,
    '4-A-5': None,
    '4-A-6': None,
    '4-A-7': None,
    '4-B-4': None,
    '4-B-5': None,
    '4-B-7': None,
    '4-B-8': None,
    '4-C-7': None,
    '5-A-3': None,
    '5-A-4': None,
    '5-A-5': None,
    '5-A-6': None,
    '5-A-7': None,
    '5-A-8': None,
    '5-B-4': None,
    '5-B-5': None,
    '5-B-6': None,
    '5-B-8': None,
    '5-C-6': None,
    '5-D-6': None,
    '6-A-3': None,
    '6-A-6': None,
    '6-A-7': None,
    '6-B-8': None,
    '6-C-6': None,
    '6-C-8': None,
    '6-D-6': None,
    '6-D-8': None,
    '7-A-4': None,
    '7-B-7': None,
    '7-C-8': None,
    '8-C-8': None,
    '8-D-8': None,
    '9-A-8': None,
    '9-D-8': None,
    '9-E-7': None
}

In [22]:
df = pd.json_normalize(all_patents)
df.head()

,url,flesh.brix_text,flesh.brix_value,flesh.color,flesh.munsell,flesh.hex,flesh.rhs,skin.color,skin.munsell,skin.hex,skin.rhs
0,https://patents.google.com/patent/USPP36186P2/en,"Brix .—Average Brix 26.9°, varies slightly wit...",26.9,Color .—Varies from 2.5Y 8/10 to 5Y 8/8.,"[2.5Y 8/10, 5Y 8/8]","[#f5c342, #e4c959]",[],Color .—Ground color varies from 7.5Y 8/6 to 1...,"[7.5Y 8/6, 10Y 8/4, 5R 2/4, 7.5R 3/8]","[#d8cc75, #d0cd91, #4d252a, #812a27]",[]
1,https://patents.google.com/patent/USPP35196P2/en,"Brix .—Average Brix 16.5°, varies slightly wit...",16.5,Color .—Varies from 5R 3/8 to 5R 3/6.,"[5R 3/8, 5R 3/6]","[#812930, #753336]",[],Color .—Ground color varies from 2.5Y 8.5/4 to...,"[5Y 8/4, 5R 3/6, 7.5R 3/6]","[#d8ca92, #753336, #75342f]",[]
2,https://patents.google.com/patent/USPP35178P2/en,"Brix .—Average Brix 19.0°, varies slightly wit...",19.0,Color .—Varies from 2.5R 4/10 to 5R 4/10.,"[2.5R 4/10, 5R 4/10]","[#a73b4d, #a83c40]",[],Color .—Ground color varies from 2.5Y 8.5/6 to...,"[2.5Y 8/6, 2.5R 3/6, 5R 2/6]","[#e7c67b, #75333c, #581d28]",[]
3,https://patents.google.com/patent/USPP34290P2/en,"Brix.— Average Brix 25.0°, varies slightly wit...",25.0,Color.— Varies from 5R 4/8 to 2.5Y 8.5/4.,[5R 4/8],[#9d4546],[],Color.— Ground color varies from 7.5Y 8/4 to 7...,"[7.5Y 8/4, 7.5Y 7/4, 5R 3/4, 7.5R 3/4]","[#d4cb91, #b9b078, #6a3b3b, #6a3b37]",[]
4,https://patents.google.com/patent/USPP34426P2/en,"Brix .—Average Brix 16.0°, varies slightly wit...",16.0,Color .—Varies from 7.5R 3/12 to 7.5R 2/8.,"[7.5R 3/12, 7.5R 2/8]","[None, #62101c]",[],Color .—Ground color varies from 7.5Y 8.5/10 t...,"[7.5Y 8/6, 7.5R 3/10, 7.5R 2/8]","[#d8cc75, #8b1d1e, #62101c]",[]


In [23]:
df.to_csv("scraped_patents.csv", index=False)